In [2]:
import pandas as pd
import json

In [44]:
doi_df =  pd.read_csv('dois_not_arxiv.csv')

In [50]:
# Get the dois_id from the doi column
doi_df['dois_id'] = doi_df['doi'].str.split('org/').str[-1]

In [52]:
# we are going to delete the ones that have figshare in the doi_id, due to the fact that they are not in the unpaywall api
doi_df = doi_df[~doi_df['dois_id'].str.contains('figshare')]

In [54]:
# we are going to delete the ones that have zenodo in the doi_id, due to the fact that they are not in the unpaywall api
doi_df = doi_df[~doi_df['dois_id'].str.contains('zenodo')]

In [55]:
# we are going to negociate pdf content with the unpaywall api
doi_df['pdf_url'] = 'https://api.unpaywall.org/v2/' + doi_df['dois_id'] + '?email=YOUR_EMAIL'

,doi,name,dois_id
4,https://doi.org/10.3389/fpls.2018.01030,openaire,10.3389/fpls.2018.01030
5,https://doi.org/10.1186/s13007-017-0199-x,openaire,10.1186/s13007-017-0199-x
6,https://doi.org/10.1111/pce.13035,openaire,10.1111/pce.13035
7,https://doi.org/10.1101/102426,openaire,10.1101/102426
8,https://doi.org/10.1093/jxb/eraa390,openaire,10.1093/jxb/eraa390
...,...,...,...
44418,https://doi.org/10.11591/ijeecs.v13.i2.pp831-836,zenodo,10.11591/ijeecs.v13.i2.pp831-836
44421,https://doi.org/10.1364/OE.24.0A1245,zenodo,10.1364/OE.24.0A1245
44429,https://doi.org/10.37871/jbres1442,zenodo,10.37871/jbres1442
44435,https://doi.org/10.35940/ijeat.C5328.029320,zenodo,10.35940/ijeat.C5328.029320


In [41]:

import requests

def download_pdf(url):
    r = requests.get(url)
    with open('arxiv_pdfs/' + url.split('/')[-1], 'wb') as f:
        f.write(r.content)

for url in df_arxiv['arxiv_url']:
    download_pdf(url)
    break